# pyResToolbox Nodal Analysis Examples

This notebook demonstrates the multi-segment nodal analysis framework in pyResToolbox, including:
- **Vertical** and **deviated/horizontal** wellbores
- **Gas** and **oil** wells
- **2, 3, and 4 segment** completions
- **Inflow (IPR)** and **outflow (VLP)** curves with operating point solutions
- Multiple VLP methods: Hagedorn-Brown (HB), Beggs & Brill (BB), Woldesemayat-Ghajar (WG)
- BUR gas PVT method (tuned 5-component Peng-Robinson EOS)

In [ ]:
import sys
sys.path.insert(0, '/home/mark/projects')

import numpy as np
import matplotlib.pyplot as plt
from pyrestoolbox import nodal, gas, oil

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

## Helper: Nodal Plot Function

A reusable function to plot IPR and VLP curves with the operating point marked.

In [ ]:
def plot_nodal(result, well_type='gas', title='Nodal Analysis'):
    """Plot IPR and VLP curves with operating point from operating_point() result."""
    fig, ax = plt.subplots()

    # Extract data
    vlp_rates = result['vlp']['rates']
    vlp_bhp = result['vlp']['bhp']
    ipr_pwf = result['ipr']['pwf']
    ipr_rates = result['ipr']['rate']
    op_rate = result['rate']
    op_bhp = result['bhp']

    # For gas wells, IPR rates are in Mscf/d but VLP rates are in MMscf/d
    if well_type == 'gas':
        ipr_rates_plot = [r / 1000.0 for r in ipr_rates]  # Convert Mscf/d -> MMscf/d
        rate_label = 'Gas Rate (MMscf/d)'
    else:
        ipr_rates_plot = ipr_rates
        rate_label = 'Liquid Rate (STB/d)'

    ax.plot(ipr_rates_plot, ipr_pwf, 'b-', linewidth=2, label='IPR')
    ax.plot(vlp_rates, vlp_bhp, 'r-', linewidth=2, label='VLP (Outflow)')
    ax.plot(op_rate, op_bhp, 'ko', markersize=10, zorder=5, label=f'Operating Point')

    if well_type == 'gas':
        ax.annotate(f'  {op_rate:.2f} MMscf/d\n  {op_bhp:.0f} psia',
                    xy=(op_rate, op_bhp), fontsize=10,
                    xytext=(op_rate + 0.5, op_bhp + 100))
    else:
        ax.annotate(f'  {op_rate:.0f} STB/d\n  {op_bhp:.0f} psia',
                    xy=(op_rate, op_bhp), fontsize=10,
                    xytext=(op_rate + 100, op_bhp + 100))

    ax.set_xlabel(rate_label, fontsize=12)
    ax.set_ylabel('Pressure (psia)', fontsize=12)
    ax.set_title(title, fontsize=13)
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    return fig

---
## Example 1: Vertical Gas Well — 2 Segments (Tubing + Casing)

A vertical gas well with 2 segments:
- 8,000 ft of 2.441" tubing
- 2,000 ft of 4.892" casing below the tubing shoe

Uses BUR gas PVT with 5% CO2.

In [ ]:
# Define GasPVT with BUR method
gpvt_1 = gas.GasPVT(sg=0.70, co2=0.05, zmethod='BUR', cmethod='BUR')
print(f'Gas Tc = {gpvt_1.tc:.1f} deg R, Pc = {gpvt_1.pc:.1f} psia')
print(f'Z @ 3000 psia, 220F = {gpvt_1.z(3000, 220):.4f}')
print(f'Viscosity @ 3000 psia, 220F = {gpvt_1.viscosity(3000, 220):.4f} cP')

In [ ]:
# 2-Segment vertical completion
segs_2v = [
    nodal.WellSegment(md=8000, id=2.441, deviation=0),   # Tubing: 8000 ft vertical
    nodal.WellSegment(md=2000, id=4.892, deviation=0),   # Casing: 2000 ft vertical
]
comp_2v = nodal.Completion(segments=segs_2v, tht=100, bht=250)
print(f'Total MD = {comp_2v.total_md} ft')
print(f'Total TVD = {comp_2v.total_tvd:.0f} ft')
print(f'Segments: {len(comp_2v.segments)}')

In [ ]:
# Reservoir
res_1 = nodal.Reservoir(pr=3500, degf=250, k=15, h=60, re=1500, rw=0.35, S=3, D=0.0005)

# Operating point
result_1 = nodal.operating_point(
    thp=500, completion=comp_2v, reservoir=res_1,
    vlpmethod='BB', well_type='gas', gsg=0.70,
    cgr=15, qw_bwpd=20, api=50, n_points=30
)
print(f"Operating Rate: {result_1['rate']:.2f} MMscf/d")
print(f"Operating BHP:  {result_1['bhp']:.0f} psia")

plot_nodal(result_1, well_type='gas',
           title='Example 1: Vertical Gas Well — 2 Segments (BB Method)');

---
## Example 2: Deviated Gas Well — 3 Segments

A deviated gas well with 3 segments:
- 5,000 ft of 2.441" tubing — vertical (0 deg deviation)
- 3,000 ft of 2.441" tubing — building angle (30 deg deviation)
- 4,000 ft of 4.892" casing — high angle (60 deg deviation)

BB and WG are suitable for deviated wells. Uses BUR gas PVT.

In [ ]:
# 3-Segment deviated completion
segs_3d = [
    nodal.WellSegment(md=5000, id=2.441, deviation=0),    # Vertical tubing
    nodal.WellSegment(md=3000, id=2.441, deviation=30),   # Build section
    nodal.WellSegment(md=4000, id=4.892, deviation=60),   # High-angle casing
]
comp_3d = nodal.Completion(segments=segs_3d, tht=90, bht=230)
print(f'Total MD = {comp_3d.total_md} ft')
print(f'Total TVD = {comp_3d.total_tvd:.0f} ft')
for i, s in enumerate(comp_3d.segments):
    print(f'  Seg {i+1}: MD={s.md} ft, ID={s.id}", dev={s.deviation} deg, TVD={s.tvd:.0f} ft')

In [ ]:
# Reservoir
res_2 = nodal.Reservoir(pr=4000, degf=230, k=20, h=50, re=2000, rw=0.35, S=2, D=0.0003)

# Operating point with BB (suitable for deviated wells)
result_2 = nodal.operating_point(
    thp=600, completion=comp_3d, reservoir=res_2,
    vlpmethod='BB', well_type='gas', gsg=0.70,
    cgr=10, qw_bwpd=15, api=50, n_points=30
)
print(f"Operating Rate: {result_2['rate']:.2f} MMscf/d")
print(f"Operating BHP:  {result_2['bhp']:.0f} psia")

plot_nodal(result_2, well_type='gas',
           title='Example 2: Deviated Gas Well — 3 Segments (BB Method)');

---
## Example 3: Highly Deviated Gas Well — 4 Segments

A highly deviated (near-horizontal) gas well with 4 segments:
- 3,000 ft of 2.441" tubing — vertical
- 2,000 ft of 2.441" tubing — building (20 deg)
- 3,000 ft of 2.441" tubing — tangent section (55 deg)
- 4,000 ft of 4.892" casing — near-horizontal (80 deg)

Uses WG method (well-suited for all inclinations) and BUR gas PVT.

In [ ]:
# 4-Segment highly deviated completion
segs_4d = [
    nodal.WellSegment(md=3000, id=2.441, deviation=0),    # Vertical
    nodal.WellSegment(md=2000, id=2.441, deviation=20),   # Build
    nodal.WellSegment(md=3000, id=2.441, deviation=55),   # Tangent
    nodal.WellSegment(md=4000, id=4.892, deviation=80),   # Near-horizontal
]
comp_4d = nodal.Completion(segments=segs_4d, tht=85, bht=220)
print(f'Total MD = {comp_4d.total_md} ft')
print(f'Total TVD = {comp_4d.total_tvd:.0f} ft')
for i, s in enumerate(comp_4d.segments):
    print(f'  Seg {i+1}: MD={s.md} ft, ID={s.id}", dev={s.deviation} deg, TVD={s.tvd:.0f} ft')

In [ ]:
# Reservoir
res_3 = nodal.Reservoir(pr=4500, degf=220, k=25, h=40, re=2000, rw=0.35, S=1, D=0.0002)

# Operating point with WG (suitable for all inclinations)
result_3 = nodal.operating_point(
    thp=500, completion=comp_4d, reservoir=res_3,
    vlpmethod='WG', well_type='gas', gsg=0.70,
    cgr=20, qw_bwpd=30, api=48, n_points=30
)
print(f"Operating Rate: {result_3['rate']:.2f} MMscf/d")
print(f"Operating BHP:  {result_3['bhp']:.0f} psia")

plot_nodal(result_3, well_type='gas',
           title='Example 3: Highly Deviated Gas Well — 4 Segments (WG Method)');

---
## Example 4: Vertical vs Deviated Comparison — Same Gas Well

Compare VLP outflow curves for the same well with vertical vs deviated trajectory. 
Same total MD (12,000 ft), but the deviated well has less TVD and therefore lower hydrostatic head.

In [ ]:
# Vertical: single segment, 12,000 ft
comp_vert = nodal.Completion(
    segments=[nodal.WellSegment(md=12000, id=2.441, deviation=0)],
    tht=100, bht=260
)

# Deviated: 3 segments totaling 12,000 ft MD
comp_dev = nodal.Completion(
    segments=[
        nodal.WellSegment(md=4000, id=2.441, deviation=0),
        nodal.WellSegment(md=4000, id=2.441, deviation=45),
        nodal.WellSegment(md=4000, id=2.441, deviation=75),
    ],
    tht=100, bht=260
)
print(f'Vertical: MD={comp_vert.total_md} ft, TVD={comp_vert.total_tvd:.0f} ft')
print(f'Deviated: MD={comp_dev.total_md} ft, TVD={comp_dev.total_tvd:.0f} ft')

In [ ]:
# Generate VLP curves for both
rates = list(np.linspace(0.5, 25, 25))

vlp_vert = nodal.outflow_curve(
    thp=500, completion=comp_vert, vlpmethod='BB', well_type='gas',
    rates=rates, gsg=0.70, cgr=10, qw_bwpd=10
)
vlp_dev = nodal.outflow_curve(
    thp=500, completion=comp_dev, vlpmethod='BB', well_type='gas',
    rates=rates, gsg=0.70, cgr=10, qw_bwpd=10
)

fig, ax = plt.subplots()
ax.plot(vlp_vert['rates'], vlp_vert['bhp'], 'r-', linewidth=2,
        label=f'Vertical (TVD={comp_vert.total_tvd:.0f} ft)')
ax.plot(vlp_dev['rates'], vlp_dev['bhp'], 'b--', linewidth=2,
        label=f'Deviated (TVD={comp_dev.total_tvd:.0f} ft)')
ax.set_xlabel('Gas Rate (MMscf/d)', fontsize=12)
ax.set_ylabel('BHP (psia)', fontsize=12)
ax.set_title('VLP Comparison: Vertical vs Deviated (Same MD, BB Method)', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## Example 5: Vertical Oil Well — 2 Segments with OilPVT

A vertical oil well with 2 segments:
- 6,000 ft of 2.992" tubing
- 2,000 ft of 5.921" casing

Uses OilPVT for consistent PVT throughout IPR and VLP. Undersaturated reservoir (Pr > Pb) so IPR uses Darcy above Pb and Vogel below.

In [ ]:
# Oil PVT
opvt = oil.OilPVT(api=32, sg_sp=0.70, pb=2800, rsb=550)
print(f'Oil SG = {opvt.sg_o:.4f}')
print(f'Rs @ 3000 psia, 200F = {opvt.rs(3000, 200):.1f} scf/stb')
print(f'Bo @ 3000 psia, 200F = {opvt.bo(3000, 200):.4f} rb/stb')
print(f'Viscosity @ 3000 psia, 200F = {opvt.viscosity(3000, 200):.4f} cP')

In [ ]:
# 2-Segment vertical oil completion
segs_oil2 = [
    nodal.WellSegment(md=6000, id=2.992, deviation=0),
    nodal.WellSegment(md=2000, id=5.921, deviation=0),
]
comp_oil2 = nodal.Completion(segments=segs_oil2, tht=120, bht=200)

# Reservoir (Pr > Pb → undersaturated)
res_oil = nodal.Reservoir(pr=3500, degf=200, k=80, h=35, re=1200, rw=0.35, S=1)

result_oil2 = nodal.operating_point(
    thp=250, completion=comp_oil2, reservoir=res_oil,
    vlpmethod='HB', well_type='oil',
    oil_pvt=opvt, gor=700, wc=0.2, gsg=0.70, n_points=30
)
print(f"Operating Rate: {result_oil2['rate']:.0f} STB/d")
print(f"Operating BHP:  {result_oil2['bhp']:.0f} psia")

plot_nodal(result_oil2, well_type='oil',
           title='Example 5: Vertical Oil Well — 2 Segments (HB Method)');

---
## Example 6: Deviated Oil Well — 3 Segments with OilPVT

A deviated oil well with 3 segments:
- 4,000 ft of 2.992" tubing — vertical
- 2,500 ft of 2.992" tubing — building angle (35 deg deviation)
- 2,500 ft of 5.921" casing — high angle (65 deg deviation)

Uses BB method (suitable for all inclinations).

In [ ]:
# 3-Segment deviated oil completion
segs_oil3 = [
    nodal.WellSegment(md=4000, id=2.992, deviation=0),
    nodal.WellSegment(md=2500, id=2.992, deviation=35),
    nodal.WellSegment(md=2500, id=5.921, deviation=65),
]
comp_oil3 = nodal.Completion(segments=segs_oil3, tht=110, bht=195)
print(f'Total MD = {comp_oil3.total_md} ft, TVD = {comp_oil3.total_tvd:.0f} ft')

# Same reservoir
res_oil3 = nodal.Reservoir(pr=3500, degf=195, k=80, h=35, re=1200, rw=0.35, S=1)

result_oil3 = nodal.operating_point(
    thp=250, completion=comp_oil3, reservoir=res_oil3,
    vlpmethod='BB', well_type='oil',
    oil_pvt=opvt, gor=700, wc=0.2, gsg=0.70, n_points=30
)
print(f"Operating Rate: {result_oil3['rate']:.0f} STB/d")
print(f"Operating BHP:  {result_oil3['bhp']:.0f} psia")

plot_nodal(result_oil3, well_type='oil',
           title='Example 6: Deviated Oil Well — 3 Segments (BB Method)');

---
## Example 7: Horizontal Oil Well — 4 Segments with OilPVT

A horizontal oil well with 4 segments:
- 3,000 ft of 2.992" tubing — vertical
- 1,500 ft of 2.992" tubing — building (25 deg)
- 1,500 ft of 2.992" tubing — tangent (60 deg)
- 3,000 ft of 4.892" casing — near-horizontal (85 deg)

Uses BB method.

In [ ]:
# 4-Segment horizontal oil completion
segs_oil4 = [
    nodal.WellSegment(md=3000, id=2.992, deviation=0),     # Vertical
    nodal.WellSegment(md=1500, id=2.992, deviation=25),    # Build
    nodal.WellSegment(md=1500, id=2.992, deviation=60),    # Tangent
    nodal.WellSegment(md=3000, id=4.892, deviation=85),    # Near-horizontal
]
comp_oil4 = nodal.Completion(segments=segs_oil4, tht=100, bht=190)
print(f'Total MD = {comp_oil4.total_md} ft, TVD = {comp_oil4.total_tvd:.0f} ft')
for i, s in enumerate(comp_oil4.segments):
    print(f'  Seg {i+1}: MD={s.md} ft, ID={s.id}", dev={s.deviation} deg, TVD={s.tvd:.0f} ft')

In [ ]:
res_oil4 = nodal.Reservoir(pr=3200, degf=190, k=100, h=30, re=1000, rw=0.35, S=0)

result_oil4 = nodal.operating_point(
    thp=200, completion=comp_oil4, reservoir=res_oil4,
    vlpmethod='BB', well_type='oil',
    oil_pvt=opvt, gor=600, wc=0.25, gsg=0.70, n_points=30
)
print(f"Operating Rate: {result_oil4['rate']:.0f} STB/d")
print(f"Operating BHP:  {result_oil4['bhp']:.0f} psia")

plot_nodal(result_oil4, well_type='oil',
           title='Example 7: Horizontal Oil Well — 4 Segments (BB Method)');

---
## Example 8: VLP Method Comparison — 3-Segment Deviated Gas Well

Compare all four VLP methods on the same deviated gas well to illustrate method sensitivity. 
Note: HB and Gray were developed for vertical flow and may be unreliable beyond ~30 deg deviation.

In [ ]:
# 3-Segment deviated gas well
segs_cmp = [
    nodal.WellSegment(md=5000, id=2.441, deviation=0),
    nodal.WellSegment(md=3000, id=2.441, deviation=25),
    nodal.WellSegment(md=3000, id=4.892, deviation=50),
]
comp_cmp = nodal.Completion(segments=segs_cmp, tht=90, bht=240)

rates_cmp = list(np.linspace(0.5, 20, 25))

fig, ax = plt.subplots()
colors = {'HB': 'red', 'BB': 'blue', 'WG': 'green', 'GRAY': 'purple'}
styles = {'HB': '-', 'BB': '--', 'WG': '-.', 'GRAY': ':'}

for method in ['HB', 'BB', 'WG', 'GRAY']:
    vlp = nodal.outflow_curve(
        thp=500, completion=comp_cmp, vlpmethod=method, well_type='gas',
        rates=rates_cmp, gsg=0.70, cgr=10, qw_bwpd=10
    )
    ax.plot(vlp['rates'], vlp['bhp'], color=colors[method],
            linestyle=styles[method], linewidth=2, label=method)

ax.set_xlabel('Gas Rate (MMscf/d)', fontsize=12)
ax.set_ylabel('BHP (psia)', fontsize=12)
ax.set_title('VLP Method Comparison — 3-Segment Deviated Gas Well', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## Example 9: Gas Well with H2 Blend — BUR Method Auto-Selection

When hydrogen is present, GasPVT automatically selects BUR/BNS for both z-factor and critical property methods. 
3-segment well with 10% H2 in the gas.

In [ ]:
# GasPVT with H2 — auto-selects BNS/BUR
gpvt_h2 = gas.GasPVT(sg=0.55, co2=0.02, h2=0.10)
print(f'Z-method: {gpvt_h2.zmethod}')
print(f'C-method: {gpvt_h2.cmethod}')
print(f'Tc = {gpvt_h2.tc:.1f} deg R, Pc = {gpvt_h2.pc:.1f} psia')
print(f'Z @ 2500 psia, 200F = {gpvt_h2.z(2500, 200):.4f}')

In [ ]:
# 3-Segment completion
segs_h2 = [
    nodal.WellSegment(md=5000, id=2.441, deviation=0),
    nodal.WellSegment(md=3000, id=2.441, deviation=30),
    nodal.WellSegment(md=2000, id=4.892, deviation=0),
]
comp_h2 = nodal.Completion(segments=segs_h2, tht=100, bht=220)

res_h2 = nodal.Reservoir(pr=3000, degf=220, k=12, h=45, re=1500, rw=0.35, S=2, D=0.0004)

result_h2 = nodal.operating_point(
    thp=400, completion=comp_h2, reservoir=res_h2,
    vlpmethod='BB', well_type='gas', gsg=0.55, n_points=30
)
print(f"Operating Rate: {result_h2['rate']:.2f} MMscf/d")
print(f"Operating BHP:  {result_h2['bhp']:.0f} psia")

plot_nodal(result_h2, well_type='gas',
           title='Example 9: Gas + 10% H2 Well — 3 Segments (BUR Auto-Selected)');

---
## Example 10: THP Sensitivity — 4-Segment Deviated Gas Well

Show how changing tubing head pressure shifts the VLP curve and changes the operating point. Uses the 4-segment highly deviated gas well from Example 3.

In [ ]:
# Reuse 4-segment completion and reservoir from Example 3
thp_values = [300, 500, 700, 900]

fig, ax = plt.subplots()
colors_thp = ['green', 'blue', 'orange', 'red']

# Plot IPR once (it doesn't depend on THP)
ipr_data = nodal.ipr_curve(res_3, well_type='gas', gsg=0.70, n_points=30)
ipr_rates_mmscf = [r / 1000.0 for r in ipr_data['rate']]
ax.plot(ipr_rates_mmscf, ipr_data['pwf'], 'k-', linewidth=2.5, label='IPR')

for thp_val, clr in zip(thp_values, colors_thp):
    result = nodal.operating_point(
        thp=thp_val, completion=comp_4d, reservoir=res_3,
        vlpmethod='WG', well_type='gas', gsg=0.70,
        cgr=20, qw_bwpd=30, api=48, n_points=30
    )
    ax.plot(result['vlp']['rates'], result['vlp']['bhp'],
            color=clr, linewidth=1.5, label=f'VLP THP={thp_val} psia')
    ax.plot(result['rate'], result['bhp'], 'o', color=clr, markersize=8, zorder=5)
    ax.annotate(f"{result['rate']:.1f}", xy=(result['rate'], result['bhp']),
                xytext=(5, 10), textcoords='offset points', fontsize=9, color=clr)

ax.set_xlabel('Gas Rate (MMscf/d)', fontsize=12)
ax.set_ylabel('Pressure (psia)', fontsize=12)
ax.set_title('THP Sensitivity — 4-Segment Deviated Gas Well (WG Method)', fontsize=13)
ax.legend(fontsize=10, loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## Example 11: Oil Well — Vertical vs Deviated IPR/VLP Comparison

Side-by-side nodal plots comparing a vertical 2-segment oil well with a deviated 4-segment oil well. Same reservoir, same PVT.

In [ ]:
# Vertical 2-segment
comp_ov = nodal.Completion(
    segments=[
        nodal.WellSegment(md=6000, id=2.992, deviation=0),
        nodal.WellSegment(md=2000, id=5.921, deviation=0),
    ],
    tht=110, bht=200
)

# Deviated 4-segment (same total MD = 8000 ft)
comp_od = nodal.Completion(
    segments=[
        nodal.WellSegment(md=3000, id=2.992, deviation=0),
        nodal.WellSegment(md=1500, id=2.992, deviation=30),
        nodal.WellSegment(md=1500, id=2.992, deviation=55),
        nodal.WellSegment(md=2000, id=5.921, deviation=75),
    ],
    tht=110, bht=200
)

res_cmp_oil = nodal.Reservoir(pr=3500, degf=200, k=80, h=35, re=1200, rw=0.35, S=1)
opvt_cmp = oil.OilPVT(api=32, sg_sp=0.70, pb=2800, rsb=550)

result_ov = nodal.operating_point(
    thp=250, completion=comp_ov, reservoir=res_cmp_oil,
    vlpmethod='BB', well_type='oil', oil_pvt=opvt_cmp,
    gor=700, wc=0.2, gsg=0.70, n_points=30
)
result_od = nodal.operating_point(
    thp=250, completion=comp_od, reservoir=res_cmp_oil,
    vlpmethod='BB', well_type='oil', oil_pvt=opvt_cmp,
    gor=700, wc=0.2, gsg=0.70, n_points=30
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

# Vertical
ax1.plot(result_ov['ipr']['rate'], result_ov['ipr']['pwf'], 'b-', lw=2, label='IPR')
ax1.plot(result_ov['vlp']['rates'], result_ov['vlp']['bhp'], 'r-', lw=2, label='VLP')
ax1.plot(result_ov['rate'], result_ov['bhp'], 'ko', ms=10, zorder=5,
         label=f"Op: {result_ov['rate']:.0f} STB/d, {result_ov['bhp']:.0f} psia")
ax1.set_xlabel('Liquid Rate (STB/d)', fontsize=12)
ax1.set_ylabel('Pressure (psia)', fontsize=12)
ax1.set_title(f"Vertical 2-Seg (TVD={comp_ov.total_tvd:.0f} ft)", fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Deviated
ax2.plot(result_od['ipr']['rate'], result_od['ipr']['pwf'], 'b-', lw=2, label='IPR')
ax2.plot(result_od['vlp']['rates'], result_od['vlp']['bhp'], 'r-', lw=2, label='VLP')
ax2.plot(result_od['rate'], result_od['bhp'], 'ko', ms=10, zorder=5,
         label=f"Op: {result_od['rate']:.0f} STB/d, {result_od['bhp']:.0f} psia")
ax2.set_xlabel('Liquid Rate (STB/d)', fontsize=12)
ax2.set_title(f"Deviated 4-Seg (TVD={comp_od.total_tvd:.0f} ft)", fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.suptitle('Oil Well: Vertical vs Deviated Nodal Comparison (BB Method)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---
## Example 12: Standalone fbhp Calls — Segment Detail

Direct `fbhp()` calls showing the effect of adding segments. Start with a simple 2-segment vertical gas well, then progressively add deviation to see the BHP reduction.

In [ ]:
# Common parameters
thp_fb = 500
qg = 5.0  # MMscf/d

# 2-Segment vertical
c2v = nodal.Completion(
    segments=[
        nodal.WellSegment(md=7000, id=2.441, deviation=0),
        nodal.WellSegment(md=3000, id=4.892, deviation=0),
    ], tht=100, bht=230
)

# 3-Segment with build
c3d = nodal.Completion(
    segments=[
        nodal.WellSegment(md=5000, id=2.441, deviation=0),
        nodal.WellSegment(md=2000, id=2.441, deviation=30),
        nodal.WellSegment(md=3000, id=4.892, deviation=45),
    ], tht=100, bht=230
)

# 4-Segment near-horizontal
c4h = nodal.Completion(
    segments=[
        nodal.WellSegment(md=4000, id=2.441, deviation=0),
        nodal.WellSegment(md=2000, id=2.441, deviation=25),
        nodal.WellSegment(md=2000, id=2.441, deviation=55),
        nodal.WellSegment(md=2000, id=4.892, deviation=80),
    ], tht=100, bht=230
)

configs = [
    ('2-Seg Vertical', c2v),
    ('3-Seg Deviated (0/30/45)', c3d),
    ('4-Seg Near-Horizontal (0/25/55/80)', c4h),
]

print(f'{"Configuration":<40s} {"MD (ft)":>8s} {"TVD (ft)":>9s} {"BHP (psia)":>11s}')
print('-' * 70)
for name, comp in configs:
    bhp = nodal.fbhp(
        thp=thp_fb, completion=comp, vlpmethod='BB', well_type='gas',
        qg_mmscfd=qg, gsg=0.70, cgr=10, qw_bwpd=10
    )
    print(f'{name:<40s} {comp.total_md:>8.0f} {comp.total_tvd:>9.0f} {bhp:>11.1f}')

---
## Summary

| Example | Well Type | Segments | Trajectory | VLP Method | Key Feature |
|---------|-----------|----------|------------|------------|-------------|
| 1 | Gas | 2 | Vertical | BB | Basic multi-segment, BUR PVT |
| 2 | Gas | 3 | Deviated | BB | Build + high-angle sections |
| 3 | Gas | 4 | Near-horizontal | WG | 4-segment trajectory, WG method |
| 4 | Gas | 1 vs 3 | Vertical vs Deviated | BB | VLP comparison overlay |
| 5 | Oil | 2 | Vertical | HB | OilPVT, undersaturated IPR |
| 6 | Oil | 3 | Deviated | BB | Deviated oil with OilPVT |
| 7 | Oil | 4 | Near-horizontal | BB | 4-segment horizontal oil |
| 8 | Gas | 3 | Deviated | HB/BB/WG/GRAY | Method comparison |
| 9 | Gas+H2 | 3 | Deviated | BB | H2 blend, BUR auto-selection |
| 10 | Gas | 4 | Deviated | WG | THP sensitivity |
| 11 | Oil | 2 vs 4 | Vertical vs Deviated | BB | Side-by-side nodal comparison |
| 12 | Gas | 2/3/4 | Progressive deviation | BB | fbhp table comparison |